In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
HERV_path = "/path/of/herv.anno.h5ad"
RNA_path = "/path/of/GSE196830_re_anno.h5ad"

In [ ]:
def adata_group(adata, groupby, layer=None, use_raw=False, min_cells=0):
    
    counts = adata.obs[groupby].value_counts() 
    indiv = counts[counts >= min_cells].index
    adata = adata[adata.obs[groupby].isin(indiv), :]
    index = adata.obs[groupby].unique()
    if use_raw:
        adata = adata.raw.to_adata()
    
    if layer is not None:
        csr = adata.layers[layer]
    else:
        csr = adata.X
    
    df = pd.DataFrame([np.array(csr[adata.obs[groupby]==x, :].sum(axis=0))[0, :] for x in index], index=index, columns=adata.var_names)
    
    bulk = sc.AnnData(df)
    
    age_dict = dict(zip(adata.obs[groupby], adata.obs['age']))
    sex_dict = dict(zip(adata.obs[groupby], adata.obs['sex']))
    age_phase_dict = dict(zip(adata.obs[groupby], adata.obs['age_phase']))
    age_stage_dict = dict(zip(adata.obs[groupby], adata.obs['age_stage']))
    bulk.obs['age'] = [age_dict[x] for x in bulk.obs_names]
    bulk.obs['sex'] = [sex_dict[x] for x in bulk.obs_names]
    bulk.obs['age_phase'] = [age_phase_dict[x] for x in bulk.obs_names]
    bulk.obs['age_stage'] = [age_stage_dict[x] for x in bulk.obs_names]
    bulk.obs[groupby] = bulk.obs_names.values
    return bulk

In [ ]:
RNA = sc.read(RNA_path)
HERV = sc.read(HERV_path)

In [ ]:
for celltype in ['CD4-T','CD8-T','Bcells','NK','Myeloid']:    
    RNA = sc.read(RNA_path)
    RNA_tmp = RNA[RNA.obs['celltype']==celltype]
    bulk_RNA = adata_group(RNA_tmp, 'indiv_ID',layer="counts")
    bulk_RNA.var['mt'] = bulk_RNA.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
    sc.pp.calculate_qc_metrics(bulk_RNA, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    HERV = sc.read(HERV_path)
    HERV_tmp = HERV[HERV.obs['celltype']==celltype]
    bulk_HERV = adata_group(HERV_tmp, 'indiv_ID')
    gene_expression = bulk_HERV.X

    total_counts = bulk_RNA.obs['total_counts'].values

    for i in range(gene_expression.shape[0]):
        gene_expression[i] = gene_expression[i] / total_counts[i] *1e6

    obs = bulk_HERV.obs.copy()
    var = bulk_HERV.var.copy()

    bulk_HERV_norm = sc.AnnData(X=gene_expression)
    bulk_HERV_norm.obs = obs
    bulk_HERV_norm.var = var

    sc.pp.log1p(bulk_HERV_norm)
    df = pd.DataFrame(data=bulk_HERV_norm.X,index=bulk_HERV_norm.obs_names,columns=bulk_HERV_norm.var_names)
    df.to_csv(f"/path/of/expression/{celltype}/{celltype}_herv.csv")